In [185]:
import random
import time
gap.eval('LoadPackage("sonata")')

def p_part_of_number(p, num):
    """
    Returns the p-part of num
    """
    # Convert p, num to int
    p, num = int(p), int(num)
    p_part = 1

    # While p-part still in num, add to p_part and divide num by p 
    while num % p == 0:
        num //= p
        p_part *= p
    return p_part


def set_maker(num):
    """ 
    Creates a set of numbers from 1 to log_2(num) based on binary representation of num
    """
    n = 1
    s = gap.Set([])

    # While num not zero, add nth number to set if num is odd, and divide num by 2
    while num != 0:
        if num % 2 == 1:
            s.AddSet(n)
        num = num // 2
        n += 1
    #print("hi im your set ", str(s))
    return s
    
    
def partition_finder(deg, ind):
    """
    For primitive permutation group defined by deg, ind, attempts to find a partition that is not half of the set
    with a stabilizer that is p' for every prime divisor p >= 5 dividing the order of the group
    
    Returns verification which tracks which primes it successfully found a partition for,
    and good_partition which returns each successful partition
    """
    
    # Generate list of numbers based on degree of group
    numbers = gap.Set([1..deg])

    # Create group and find its order
    G = gap.PrimitiveGroup(deg,ind)
    order = int(G.Order())

    # Find prime divisors of group >= 5
    prime_divisors = [p for p in map(int, gap.PrimeDivisors(order)) if p >= 5]
    #print("prime_divisors is :" + str(prime_divisors))
    verification = {p: False for p in prime_divisors}
    good_partitions = []

    n = 0
    # Finds partitions according to specification (randomly generates partitions using partition finder)
    for i in range(2**(deg-1)):
        random_index = random.randint(1, 2**(deg))
        #print(random_index)
        partition = set_maker(random_index)
        partition_size = gap.Size(partition)
        if partition_size == deg//2 or partition_size == 0 or partition_size == deg :
            continue
        stab1 = gap.Stabilizer(G, partition, "OnSets")
        size = gap.Size(stab1)
        #print("I'm stab1 ", stab1)
        #print("stab1 is size ", size)
        n += 1
        for p in prime_divisors:
            if p_part_of_number(p, size) == 1:
                verification[p] = True
                good_partitions.append(partition)
                prime_divisors.remove(p)
                break
        #print(len(prime_divisors))
        if len(prime_divisors) == 0:
            return verification, good_partitions

    #Return results
    return verification, good_partitions



def half_size_checker(partitions, order):
    """ 
    Verifies if no partitions in a set of partitions are half the size of order
    """
    order = int(order)
    for orbit in orbits:
        if len(orbit[1]) == order/2:
            print(orbit)
            return false
    return true

def num_group_deg(deg):
    """
    Returns out the number of primitive permutation groups of degree deg
    """
    n = 1
    while true:
        try: 
            gap.PrimitiveGroup(deg,n)
            n += 1
        except:
            n -= 1
            #print("I did num_group_deg")
            return n

def multi_tester(deg, max_ind):
    """
    Given degree deg and the maximum index for primitive permutation groups of degree deg, 
    tests and prints results for all groups of degree deg
    """
    for i in range(1, max_ind + 1):
        verification_results, good_partitions = partition_finder(deg, i)
        print("Verification of all primes:", verification_results)
        print("Detailed results for each prime:", verification_results)
        print(good_partitions)

def single_tester(deg, ind):
    """
    For primitive permutation group of degree deg and index ind, tests and prints results from partition_finder
    """
    verification_results, good_partitions = partition_finder(deg, ind)
    print("Primitive permutation group of degree", deg, "and index", ind)
    print("Verification of all primes:", verification_results)
    print(good_partitions)

def socle_checker(min_deg, max_deg, socle):
    """
    For group socle, checks all primitive permutation groups of degree deg \in [min_deg, max_deg] and performs half-size test on all
    groups with group.Socle() equal to socle
    """
    for n in range (min_deg, max_deg + 1):
        for m in range (1, num_group_deg(n) + 1):
            if gap.IsomorphismGroups(gap.PrimitiveGroup(n, m).Socle(), socle) != gap.IsomorphismGroups(CyclicPermutationGroup(1), CyclicPermutationGroup(2)):
                single_tester(n, m)

def socle_checker_alt(min_deg, max_deg, deg):
    """
    For degree deg, and its corresponding alternating group A_deg checks all primitive permutation groups
    of degree deg \in [min_deg, max_deg] and performs half-size test on all groups with group.Socle() equal to A_deg
    """
    socle = AlternatingGroup(deg)
    for n in range (min_deg, max_deg + 1):
        for m in range (1, num_group_deg(n) + 1):
            if gap.IsomorphismGroups(gap.PrimitiveGroup(n, m).Socle(), socle) != gap.IsomorphismGroups(CyclicPermutationGroup(1), CyclicPermutationGroup(2)) and n != deg:
                single_tester(n, m)


In [25]:
single_tester(23, 5)

Primitive permutation group of degree 23 and index 5
Verification of all primes: {5: True, 7: True, 11: True, 23: True}
Detailed results for each prime: {5: True, 7: True, 11: True, 23: True}
[[ 1, 3, 4, 5, 6, 7, 9, 11, 12, 13, 15, 19, 20, 22 ], [ 1, 6, 7, 8, 9, 10, 15, 16, 17, 18, 20, 21, 22, 23 ], [ 1, 2, 3, 5, 8, 13, 14, 15, 18, 20 ], [ 3, 4, 5, 7, 8, 13, 14, 15, 19, 23 ]]


In [15]:
n = 10
multi_tester(n, num_group_deg(n))

I did num_group_deg
prime_divisors is :[5]
Verification of all primes: {5: True}
Detailed results for each prime: {5: True}
[[ 2, 4, 8, 10 ]]
prime_divisors is :[5]
Verification of all primes: {5: True}
Detailed results for each prime: {5: True}
[[ 2, 4, 7 ]]
prime_divisors is :[5]
Verification of all primes: {5: True}
Detailed results for each prime: {5: True}
[[ 2, 3, 5, 6, 7, 8, 9, 10 ]]
prime_divisors is :[5]
Verification of all primes: {5: True}
Detailed results for each prime: {5: True}
[[ 1, 2, 5, 8, 9, 10 ]]
prime_divisors is :[5]
Verification of all primes: {5: True}
Detailed results for each prime: {5: True}
[[ 1, 5, 6, 7 ]]
prime_divisors is :[5]
Verification of all primes: {5: True}
Detailed results for each prime: {5: True}
[[ 1, 3, 4, 7, 9, 10 ]]
prime_divisors is :[5]
Verification of all primes: {5: True}
Detailed results for each prime: {5: True}
[[ 2, 3, 5, 6 ]]
prime_divisors is :[5, 7]
Verification of all primes: {5: False, 7: True}
Detailed results for each prime: {

In [241]:
min = 127
max = 290
socle = PSL(7,2)
socle_checker(min, max, socle)

Primitive permutation group of degree 127 and index 13
Verification of all primes: {5: True, 7: True, 31: True, 127: True}
[[ 1, 2, 4, 5, 6, 9, 11, 13, 15, 19, 21, 24, 26, 27, 29, 32, 34, 36, 38, 39, 
  41, 43, 48, 50, 51, 53, 55, 57, 61, 63, 64, 66, 67, 68, 69, 71, 74, 76, 80, 
  82, 84, 85, 87, 91, 92, 93, 94, 97, 98, 100, 104, 107, 109, 110, 111, 113, 
  114, 115, 118, 119, 120, 123, 126, 127 ], [ 1, 2, 4, 5, 7, 8, 10, 13, 15, 17, 18, 21, 25, 26, 27, 29, 30, 31, 35, 37, 
  38, 40, 42, 43, 46, 47, 49, 50, 51, 55, 59, 61, 62, 63, 64, 65, 67, 69, 70, 
  72, 75, 79, 81, 82, 84, 85, 86, 87, 88, 89, 93, 94, 95, 96, 102, 103, 104, 
  107, 109, 115, 116, 117, 119, 120, 121, 123, 124, 125, 127 ], [ 1, 3, 6, 8, 9, 11, 13, 20, 22, 24, 27, 28, 29, 30, 32, 34, 38, 41, 42, 43, 
  47, 49, 50, 55, 56, 57, 59, 62, 63, 64, 65, 72, 73, 74, 78, 79, 80, 83, 84, 
  85, 89, 90, 93, 95, 97, 99, 100, 104, 106, 107, 109, 110, 117, 121, 124, 
  126, 127 ], [ 2, 5, 12, 15, 21, 22, 26, 29, 30, 31, 37, 38, 40, 4

In [242]:
deg = 6
min = 15
max = 37
socle_checker_alt(min, max, deg)

os.system(f'say -v {"Thomas"} {"le programme a fini"}')


Primitive permutation group of degree 15 and index 2
Verification of all primes: {5: True}
[[ 1, 4, 6, 10, 12, 15 ]]
Primitive permutation group of degree 15 and index 3
Verification of all primes: {5: True}
[[ 1, 5, 6, 10, 13, 14 ]]
Primitive permutation group of degree 36 and index 3
Verification of all primes: {5: True}
[[ 4, 5, 6, 14, 16, 18, 21, 22, 26, 28, 30, 32, 33, 34, 35, 36 ]]
Primitive permutation group of degree 36 and index 4
Verification of all primes: {5: True}
[[ 4, 6, 7, 12, 16, 17, 20, 21, 23, 24, 25, 26, 27, 28, 29, 30, 31, 33, 34 ]]
Primitive permutation group of degree 36 and index 5
Verification of all primes: {5: True}
[[ 2, 3, 7, 10, 11, 20, 22, 23, 24, 27, 29, 32, 33, 36 ]]


0